# Installation et imports

### Installation des modules 

In [418]:
%pip install Pandas


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [419]:
%pip install Matplotlib 


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [420]:
%pip install Scikit-learn


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [421]:
%pip install Numpy


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Import et données

In [422]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib as plt
import math


In [423]:
data = pd.read_csv("../df.csv")

# Répartition des données

### Répartition des données en base d’apprentissage et de tests en utilisant les algorithmes holdout 

In [424]:
from sklearn.model_selection import train_test_split

In [425]:
def holdout(dataframe, colonne_predict, num_rep, train_s, test_s ):
        liste_data_train_test = []
        liste_data = []
        dataframe = dataframe.sample(n=7000) 
        clean_data = dataframe.drop(colonne_predict, axis = 1)
        y = dataframe[colonne_predict]
        for i in range(num_rep):
                X_train, X_test, y_train, y_test = train_test_split(clean_data,y, train_size= train_s, test_size = test_s)
                X_train = X_train.reset_index(drop=True)
                y_train = y_train.reset_index(drop=True)
                X_test = X_test.reset_index(drop=True)
                y_test = y_test.reset_index(drop=True)
                liste_data_train_test.append([X_train, X_test, y_train, y_test])
        return liste_data_train_test

In [426]:
liste_data_holdout = []
data_test = data[['descr_grav', 'latitude', 'longitude']]
liste_data_holdout  = holdout(data_test,"descr_grav", 5, 0.8, 0.2)

In [427]:
from sklearn.model_selection import LeaveOneOut
from sklearn.neighbors import KNeighborsClassifier

In [428]:
def leaveoneout(dataframe, colonne_predict):
    liste_data = []
    data = dataframe.sample(n=7000) #on prend 10% du dataframe
    data = data.reset_index(drop=True)
    X = data.drop(colonne_predict, axis = 1) 
    y = data[colonne_predict]
    LOO = LeaveOneOut()
    knn = KNeighborsClassifier(n_neighbors=5)
    
    for train_index, test_index in (LOO.split(X)):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        liste_data.append(y_pred)
        
    return liste_data

In [429]:
liste_y_pred = leaveoneout(data_test, "descr_grav")
print(liste_y_pred)

[array([0]), array([2]), array([0]), array([2]), array([0]), array([1]), array([0]), array([0]), array([1]), array([1]), array([1]), array([1]), array([1]), array([0]), array([0]), array([1]), array([1]), array([1]), array([0]), array([0]), array([0]), array([0]), array([0]), array([1]), array([2]), array([0]), array([1]), array([0]), array([1]), array([0]), array([1]), array([0]), array([0]), array([1]), array([1]), array([0]), array([0]), array([1]), array([0]), array([0]), array([0]), array([1]), array([0]), array([2]), array([0]), array([1]), array([2]), array([0]), array([1]), array([0]), array([1]), array([0]), array([1]), array([1]), array([0]), array([1]), array([0]), array([1]), array([0]), array([0]), array([1]), array([0]), array([2]), array([1]), array([1]), array([1]), array([1]), array([1]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([1]), array([0]), array([0]), array([0]), array([0]), array([1]), array([0]), array([2]), array([1]), arr

### Classification avec KNN

In [430]:
#on va prendre en entrée une ligne du df
def KnnClassification(ligne,y_train,df,k):
    dictList = [None] * len(df)
    liste_index = []
    liste_y_pred = []
    for index, row in df.iterrows():
        dictList[index] = np.linalg.norm(ligne - row)
    
    sorted_values = sorted(enumerate(dictList), key=lambda x: x[1])[:k]
    for index,value in sorted_values:
        #firstk.append([df.loc[index]])
        liste_y_pred.append(y_train[index])
    
    y_pred = np.bincount(liste_y_pred).argmax()
        

    return y_pred

In [431]:
knn = KNeighborsClassifier(n_neighbors=5)
liste_data_y_pred = []
for i in range(len(liste_data_holdout)):
    X_train = liste_data_holdout[i][0]
    y_train = liste_data_holdout[i][2]
    X_test = liste_data_holdout[i][1]
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    liste_data_y_pred.append(y_pred)
print(liste_data_y_pred)

[array([0, 0, 1, ..., 0, 0, 0]), array([0, 1, 0, ..., 1, 1, 0]), array([0, 1, 1, ..., 0, 0, 2]), array([0, 2, 0, ..., 1, 0, 1]), array([2, 2, 0, ..., 0, 0, 2])]


In [432]:
#from scratch knn

liste_y_pred = []

X_test = liste_data_holdout[0][1]
y_train  = liste_data_holdout[0][2]
X_train = liste_data_holdout[0][0]


for i in range(len(X_test)):
    y_pred = KnnClassification(X_test.loc[i], y_train, X_train, 5)
    liste_y_pred.append(y_pred)

print(liste_y_pred)

[2, 0, 1, 1, 0, 1, 2, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 2, 1, 0, 0, 0, 0, 0, 2, 1, 0, 0, 1, 0, 1, 2, 0, 0, 2, 2, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 2, 1, 1, 1, 1, 0, 0, 1, 0, 2, 0, 0, 0, 1, 0, 0, 1, 0, 2, 0, 1, 1, 1, 1, 2, 1, 0, 0, 1, 0, 2, 1, 0, 1, 1, 2, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 2, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 2, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 2, 1, 1, 1, 0, 0, 1, 2, 0, 0, 0, 1, 2, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 2, 0, 1, 0, 2, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 2, 0, 1, 0, 0, 2, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 2, 1, 2, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 2, 0, 2, 2, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 2, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 2, 0, 0, 0, 1, 2, 0, 1, 1, 0, 1, 1, 1, 1, 0, 2, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 2, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 2, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 